In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-core langchain_openai python-dotenv langsmith pydantic spotipy

In [ ]:
%pip install --quiet -U jupyterlab-lsp
%pip install --quiet -U "python-lsp-server[all]"

In [ ]:
## Setup logging
import logging
import os
from dotenv import load_dotenv

load_dotenv(override=True)
logger = logging.getLogger(__name__)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',  # Define the format
    handlers=[logging.StreamHandler()]  # Output to the console
)

# Define Spotify URI type

In [ ]:
from typing import List, NewType

# Define a custom URI type
SpotifyURI = NewType('SpotifyURI', str)

# Define Spotify Track and Playlist Models

In [ ]:
from typing import List, Dict, Optional
from pydantic import BaseModel, Field

# Define a Pydantic model for Playlist
class Playlist(BaseModel):
    """
    A model representing a Spotify playlist.
    """
    id: str = Field(..., description="The unique identifier for the playlist.")
    uri: str = Field(..., description="The Spotify URI for the playlist.")
    name: str = Field(..., description="The name of the playlist.")
    description: Optional[str] = Field(None, description="The playlist's description.")
    owner: Optional[str] = Field(None, description="The display name of the playlist's owner.")
    tracks_total: Optional[int] = Field(None, description="The total number of tracks in the playlist.")
    is_public: Optional[bool] = Field(None, description="Indicates if the playlist is public.")
    collaborative: Optional[bool] = Field(None, description="Indicates if the playlist is collaborative.")
    snapshot_id: Optional[str] = Field(None, description="The version identifier for the current playlist.")

# Define a Pydantic model for Track
class Track(BaseModel):
    """
    A model representing a Spotify track.
    """
    id: str = Field(..., description="The unique identifier for the track.")
    uri: str = Field(..., description="The Spotify URI for the track.")
    name: str = Field(..., description="The name of the track.")
    artists: List[str] = Field(..., description="A list of artists who performed the track.")
    album: str = Field(..., description="The name of the album the track is from.")
    duration_ms: Optional[int] = Field(None, description="The track length in milliseconds.")
    explicit: Optional[bool] = Field(None, description="Indicates if the track has explicit content.")
    popularity: Optional[int] = Field(None, description="The popularity of the track (0-100).")


# Define a Pydantic model for Tracks
class Tracks(BaseModel):
    """
    A model representing a collection of Spotify tracks.
    """
    tracks: List[Track] = Field(..., description="A list of Track objects.")
    total: Optional[int] = Field(None, description="Total number of tracks in the collection.")
    playlist_name: Optional[str] = Field(None, description="Name of the playlist.")

    class Config:
        """
        Configuration for the Tracks model.
        """
        str_strip_whitespace = True

In [ ]:
from typing import Dict, List, Any, Annotated, Set
from typing import Annotated
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages


class State(TypedDict, total=False):
    """
    Represents the state of the conversation and Spotify information

    Attributes:
        playlists (List[Playlist]): A list of Spotify Playlists.
        tracks (List[Track]): Track list for a Spotify Playlist
        new_playlist: (Playlist) : New Spotify playlist data
        new_tracks: (List[Track]): Tracks for the new playlist
    """
    new_playlist: Playlist
    new_tracks: List[Track]
    valid_artists: Set[str]
    candidate_artists : Set[str]
    playlists: List[Playlist]
    tracks: List[Track]
    artists: Set[str]
    messages: Annotated[List[BaseMessage], add_messages]

state: State = State()

def get_state():
    return state

# Define Spotify Tools

In [ ]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from langchain_core.tools import tool
from typing import List, NewType, Set
import json


def get_spotify_client() -> spotipy.Spotify:
    """
    Initializes and returns a Spotify client with user authentication.

    Returns:
        spotipy.Spotify: An authenticated Spotify client.
    """
    auth_manager = SpotifyClientCredentials(
        client_id=os.environ.get("SPOTIFY_CLIENT_ID"),
        client_secret=os.environ.get("SPOTIFY_CLIENT_SECRET"),
    )
    return spotipy.Spotify(auth_manager=auth_manager)


@tool
def get_playlists() -> Dict[str, List[Playlist]]:
    """
    Retrieves a list of the user's Spotify playlists.

    Returns:
        Dict[str, List[Playlist]]: A dictionary containing a list of Playlist models under the 'playlists' key.
    """
    sp = get_spotify_client()
    playlists: List[Playlist] = []

    try:
        # Fetch the current user's playlists with pagination
        playlists_raw = sp.user_playlists(user=os.getenv("SPOTIFY_USER_ID"), limit=100)
        while playlists_raw:
            for playlist_data in playlists_raw['items']:
                # Map API data to the Playlist model
                playlist = Playlist(
                    id=playlist_data['id'],
                    uri=playlist_data['uri'],
                    name=playlist_data['name'],
                    description=playlist_data.get('description'),
                    owner=playlist_data['owner']['display_name'],
                    tracks_total=playlist_data['tracks']['total'],
                    is_public=playlist_data.get('public'),
                    collaborative=playlist_data.get('collaborative'),
                    snapshot_id=playlist_data.get('snapshot_id')
                )
                playlists.append(playlist)
            # Check if there is a next page
            if playlists_raw['next']:
                playlists_raw = sp.next(playlists_raw)
            else:
                break
    except spotipy.SpotifyException as e:
        return {"error": str(e)}

    # Save state
    state: State= get_state()
    state["playlists"] = playlists

    # Serialize the playlists to JSON-serializable dictionaries
    serialized_playlists = [playlist.model_dump() for playlist in playlists]
    return {"playlists": serialized_playlists}


@tool
def get_track_list(playlist_id: SpotifyURI) -> Dict[str, List[Track]]:
    """
    Retrieves the track list for a specific Spotify playlist.

    Args:
        playlist_id (SpotifyURI): Spotify playlist URI in the format spotify:playlist:<base-62 number>

    Returns:
        Dict[str, List[Track]]: A dictionary containing a list of Track models under the 'tracks' key.
    """
    sp = get_spotify_client()
    playlist_tracks: List[Track] = []
    playlist_artists: Set[str] = set()

    try:
        # Fetch the playlist's tracks with pagination
        playlist = sp.user_playlist(user=os.getenv("SPOTIFY_USER_ID"), playlist_id=playlist_id)
        if 'tracks' in playlist:
            tracks = playlist["tracks"]
            while tracks:
                for item in tracks['items']:
                    track_data = item['track']
                    # Map API data to the Track model
                    track = Track(
                        id=track_data['id'],
                        uri=track_data['uri'],
                        name=track_data['name'],
                        artists=[artist['name'] for artist in track_data['artists']],
                        album=track_data['album']['name'],
                        duration_ms=track_data.get('duration_ms'),
                        explicit=track_data.get('explicit'),
                        popularity=track_data.get('popularity')
                    )
                    playlist_tracks.append(track)
                    [playlist_artists.add(artist['name']) for artist in track_data['artists']]
                # Check if there is a next page
                if tracks['next']:
                    # TODO unclear in this case
                    tracks = sp.next(tracks)
                else:
                    break
    except spotipy.SpotifyException as e:
        return {"error": str(e)}


    # Save state
    state: State= get_state()
    state["tracks"] = playlist_tracks
    state["artists"] = playlist_artists

    # Serialize the tracks to JSON-serializable dictionaries
    serialized_tracks = [track.model_dump() for track in playlist_tracks]
    return {"tracks": serialized_tracks}


@tool
def create_spotify_playlist(name: str, description: str = "Agentic Playlist") -> Dict[str, Playlist]:
    """
    Creates a new playlist on Spotify.

    This function only creates the playlist; it does not add tracks.
    See tool add_tracks_to_playlist() to add tracks to a existing playlist

    Args:
        name (str): The name of the new playlist.
        description (str, optional): The description of the playlist.

    Returns:
        Dict[str, Playlist]: A dictionary containing the new Playlist model under the 'new_playlist' key.
    """
    sp = get_spotify_client()
    try:
        # Create a new playlist
        new_playlist_data = sp.user_playlist_create(
            user=os.getenv("SPOTIFY_USER_ID"),
            name=name,
            public=True,
            description=description
        )
        # Map API data to the Playlist model
        new_playlist = Playlist(
            id=new_playlist_data['id'],
            uri=new_playlist_data['uri'],
            name=new_playlist_data['name'],
            description=new_playlist_data.get('description'),
            owner=new_playlist_data['owner']['display_name'],
            tracks_total=new_playlist_data['tracks']['total'],
            is_public=new_playlist_data.get('public'),
            collaborative=new_playlist_data.get('collaborative'),
            snapshot_id=new_playlist_data.get('snapshot_id')
        )
        state: State = get_state()
        state["new_playlist"] = new_playlist
    except spotipy.SpotifyException as e:
        return {"error": str(e)}
    return {"new_playlist": new_playlist.model_dump()}

@tool
def add_tracks_to_playlist(playlist_id: str, tracks: Tracks) -> Dict[str, Any]:
    """
    Adds tracks to a Spotify playlist.

    Args:
        playlist_id (str): ID of the playlist.
        tracks (Tracks): Spotify Pydantic Model of a List of tracks

    Returns:
        Dict[str, Any]: A dictionary indicating success or error.
    """
    sp = get_spotify_client()
    try:
        sp.playlist_add_items(playlist_id=playlist_id, items=tracks)
    except spotipy.SpotifyException as e:
        return {"error": str(e)}
    return {"success": True} 

@tool
def check_artists(playlist_id: str, artists: List[str]) -> Dict[str, List[str]]:
    """
    Check a list of artists for a new Playlist against a existing Playlist

    Args:
        playlist_id (SpotifyURI): Spotify playlist URI in the format spotify:playlist:<base-62 number>
        artists (str): A list of new artist names

    Returns:
        Dict[str, Set[str]]: Dict with a Set of artists that can be used in the new playlist
    """
    state: State = get_state()
    state['candidate_artists'] = set(artists)
    valid_artists = state['candidate_artists'] - state["artists"]
    state["valid_artists"] = valid_artists
    return {"valid_artists": valid_artists}



# Define Search Tool

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(
    max_results=50,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # search_depth="advanced",
    # include_domains = []
    # exclude_domains = []
)
name = search_tool.get_name()
desc = search_tool.description
desc

# Define Plan Model

In [ ]:
class Step(BaseModel):
    """
    The `Step` class represents an atomic unit of work within a plan. Each task must be focused on a single, clear objective and should not encompass multiple, unrelated activities.

    Key Points:
    - A task should perform one specific function and should not combine different activities. 
    - Each task should have a clear, verifiable success criterion that is achievable through the task's activities alone.
    - Steps must be URL-friendly, without spaces or special characters in their names.
    
    **Examples of What a Task Should NOT Do:**
    - "Create a playlist and add tracks" (This task mixes playlist creation with track management.)
    - "Search new artists and apply filtering." (This task combines artist search with its management, which should be separated.)
    - "Collect all relevant information about the playlist Spotify and User" (This task has two action items)

    Attributes:
    - `name` (str): URL-friendly name for the task, e.g., 'deploy-application'. Must not contain spaces or special characters.
    - `description` (str): A detailed description of the task, e.g., 'Deploy the application to the Kubernetes cluster using Helm.'
    - `success_criteria` (str): Clear and verifiable criteria for task completion, e.g., 'Application is successfully deployed and running in the Kubernetes cluster, as verified by checking the deployment status with 'kubectl get deployments'.' Each task must be atomic and focus on one thing only.
    - `tools` (List[str]): List of `Tool Calling` instances required for the task, specifying which tools are needed.

    """    
    name: str = Field(..., description="URL-friendly name for the task, e.g., 'deploy-application'. Must not contain spaces or special characters.")
    description: str = Field(..., description="A detailed description of the task, e.g., 'Deploy the application to the Kubernetes cluster using Helm.'")
    success_criteria: str = Field(..., description="Clear and verifiable criteria for task completion, e.g., 'Application is successfully deployed and running in the Kubernetes cluster, as verified by checking the deployment status with 'kubectl get deployments'.' Each task must be atomic and focus on one thing only.")
    tools: List[str] = Field(..., description="List of `Tool` instances required for the task, specifying which tools are needed.")

    class Config:
        """Configuration for the Task model."""
        json_schema_extra = {
            "description": "The `Task` class represents an atomic unit of work that involves using specific tools to achieve a clear goal."
            # Removed 'examples' as it is now handled at the field level
        }


class Plan(BaseModel):
    steps: List[Step] = Field(
        ..., 
        description=(
            "A list of `Steps` that constitute the plan. Each step must be atomic, "
            "focused on a specific goal, and the entire plan must be complete, verifiable, "
            "and reproducible."
        )
    )
    Reasoning: str = Field(
        ..., 
        description=(
            "OpenAI should capture here the entire reasoning sequence used to create "
            " the plan"
        )
    )

In [ ]:
@tool
def validate_plan(plan: Plan) -> bool:
    """
    Validates a step-by-step plan to solve a problem

    Args:
        plan (Plan): a step-by-step plan to solve a problem

    Returns:
        bool: whether plan is okay or not
    """
    return True

# Create ToolNode

In [ ]:
from langgraph.prebuilt import ToolNode

tools = [get_playlists, get_track_list, create_spotify_playlist, add_tracks_to_playlist, check_artists, validate_plan, search_tool]
# tools = [get_playlists, get_track_list, create_spotify_playlist, add_tracks_to_playlist, search_tool]
tool_node = ToolNode(tools)

# Manual Test of Tool Infra

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "get_playlists",
            "args" : {},
            "id": "tool_call_id",
            "type": "tool_call",
        }
    ],
)

# tool_node.invoke({"messages": [message_with_single_tool_call]})

# Bind Tools to Model

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME"), temperature=0.8)
llm_with_tools = llm.bind_tools(tools)

# Define Prompt

In [ ]:
prompt = """
Create and execute a step-by-step plan with atomic tasks to solve the following problem:

**Objective:** Build a Spotify playlist with tracks that have the same vibe and genres as my 'New Rock and Blues' playlist, following the rules below.

**Rules:**

1. **For each artist in the 'New Rock and Blues' playlist, find 3-4 artists of the similar music style**

2 .**Exclude Artists that are part of the  'New Rock and Blues' Playlist:**

3. **Focus on Post-2010 Success:** Only include tracks from artists who achieved success after the year 2010.

4. **Minimum Number of New Artists:** Include tracks from at least 40 different artists not present in the 'New Rock and Blues' playlist.

5. **For each new artist, use your Knowledge Base to recommend 3-4 tracks that match the vibe and genres.

6. **Arrange for Smooth Listening Experience:** Organize the tracks to create a smooth listening experience, considering tempo, energy, and mood, using best practices.

7. **Playlist Length:** Ensure the new playlist contains at least 100 tracks.

**Instructions:**

- Follow the rules strictly to meet the objective.
- Every step must have a single atomic action and outcome
- Prefer using a tool if available instead of your Knowledge Base
- clearly state at all times what step you are working on
- Step(s) should be repeated any number of times to achieve the desired outcome.
- Do not ask for user confirmation
"""


# First Message

In [ ]:
from langchain_core.messages import HumanMessage

human_message = HumanMessage(prompt)
ai_tool_call_message = llm_with_tools.invoke([human_message])

In [ ]:
ai_tool_call_message.pretty_print()

# chat_prompt_template holds all messages (Human, AI, Tool)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt_template: ChatPromptTemplate = human_message + ai_tool_call_message
chat_prompt_template.format_messages()

# Check for Tools in Message

In [ ]:
from langchain_core.messages import ToolMessage

tools_in_ai_message = []
tools_by_name = {tool.name: tool for tool in tools}
messages = chat_prompt_template.format_messages()
for tool_call in messages[-1].tool_calls:
    tool = tools_by_name[tool_call["name"]]
    tools_in_ai_message.append(tool)
    # observation = tool.invoke(tool_call["args"])
    # result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
if len(tools_in_ai_message) == 0 or len(tools_in_ai_message) > 1:
    raise ValueError
else:
    print(tools_in_ai_message)

# Tool Call (should be validate_plan())

In [ ]:
messages = chat_prompt_template.format_messages()
response = tool_node.invoke({"messages": messages})
tool_message = response["messages"][0]
tool_message.pretty_print()

In [ ]:
chat_prompt_template += tool_message
chat_prompt_template.format_messages()

# Send tool result to LLM

In [ ]:
ai_tool_call_message = llm_with_tools.invoke(chat_prompt_template.format_messages())
ai_tool_call_message.pretty_print()

# Tool Call (should be get_playlists())

In [ ]:
chat_prompt_template += ai_tool_call_message
messages = chat_prompt_template.format_messages()
response = tool_node.invoke({"messages": messages})
response

In [ ]:
playlist_tool_message: ToolMessage = response["messages"][0]
playlist_tool_message.pretty_print()

In [ ]:
# We need to check that is JSON serializable
# json.loads(playlist_tool_message.content)

# Format Message List for LLM

In [ ]:
chat_prompt_template += playlist_tool_message
chat_prompt_template.format_messages()

# Send Playlists Results to LLM

In [ ]:
ai_tool_call_message = llm_with_tools.invoke(chat_prompt_template.format_messages())
ai_tool_call_message.pretty_print()

In [ ]:
chat_prompt_template += ai_tool_call_message
messages = chat_prompt_template.format_messages()

# Tool Call (get_track_list)

In [ ]:
response = tool_node.invoke({"messages": messages})
response

# Decode AI Tool Call message

In [ ]:
track_list_tool_message: ToolMessage = response["messages"][0]
track_list_tool_message.pretty_print()

In [ ]:
# json.loads(track_list_tool_message.content)

# Create Track List ToolMessage for LLM

In [ ]:
# We recreate the ToolMessage to take care of UNicode characters
tool_message = ToolMessage(content=track_list_tool_message.content, name=track_list_tool_message.name, tool_call_id=track_list_tool_message.tool_call_id)
tool_message.pretty_print()

# Add Tool message containing playlist tracks to list of messages

In [ ]:
chat_prompt_template += tool_message

In [ ]:
chat_prompt_template.format_messages()

# Send Tool Result to LLM

In [ ]:
# llm_with_structured_output = llm.with_structured_output(schema=Tracks, method='function_calling', include_raw=True)
# response = llm_with_structured_output.invoke(chat_prompt_template.format_messages())
# response
response = llm_with_tools.invoke(chat_prompt_template.format_messages())
response

In [ ]:
response.pretty_print()

In [ ]:
chat_prompt_template += response
chat_prompt_template.format_messages()

# Tool Call (check_artists)

In [ ]:
response = tool_node.invoke({"messages": [response]})

In [ ]:
check_artist_tool_message: ToolMessage = response["messages"][0]
check_artist_tool_message.pretty_print()

In [ ]:
# We recreate the ToolMessage to take care of UNicode characters
tool_message = ToolMessage(content=check_artist_tool_message.content, name=check_artist_tool_message.name, tool_call_id=check_artist_tool_message.tool_call_id)
tool_message.pretty_print()

In [ ]:
chat_prompt_template += tool_message
chat_prompt_template.format_messages()

In [ ]:
response = llm_with_tools.invoke(chat_prompt_template.format_messages())

In [ ]:
response.pretty_print()

In [ ]:
chat_prompt_template += response
chat_prompt_template.format_messages()

In [ ]:
response = tool_node.invoke({"messages": [response]})

In [ ]:
response